# cancer_rename

> Please add a description. Don't forgot to edit the below cell!!

In [ ]:
#| default_exp cancer_maintrain

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [ ]:
#| hide

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():
    import os
    from google.colab import drive
    drive.mount('/content/drive')
    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')
    os.chdir('cancer-proj')
    os.system('pip install -e .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [ ]:
#| hide

from nbdev.showdoc import *

## Predictions given xval and yval:

In [ ]:
#| hide
from fastai.vision.all import *
#| export
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *

## Note on what properties we want:

- Extensible
- Easily patch in modifications etc

For now just get the data however it works. Will fix this later. But want to get `main_train` working for now

In [ ]:
#colab
train_dir='skin_cancer_ISIC/Train'
test_dir='skin_cancer_ISIC/Test'

In [ ]:
train_dir = '/content/drive/My Drive/skin_cancer_ISIC/Train'
test_dir='/content/drive/My Drive/skin_cancer_ISIC/Test'

In [ ]:
%cd ..

/content


## Load the data

In [ ]:
#| hide

#How we want to use

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
bs=256
bs_tune=256
size=128
bs_val=174
#can define other hyperparameters here
train_dir = colab_train_dir
test_dir = colab_test_dir

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders and other stuff 
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

## Load aug pipelines here

/content


## Define the augmentation pipelines

## TODO: Can probably put this in `dataloading`. No need to clog this notebook up.

In [ ]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

###################
#batch sizes, number of epochs (for training BT and for fine tuning)


###################

###################
#Hyperparams for BT augmentations
#ImageNet style augmentations (see BYOL paper)

flip_p1 = 0.5
flip_p2 = 0.5

##byol defaults
jitter_p1 = 0.8
jitter_p2 = 0.8

#byol defaults
bw_p1 = 0.2
bw_p2 = 0.2

# bw_p1 = 0.0
# bw_p2 = 0.0

blur_p1 = 1.0
blur_p2 = 0.1

sol_p1 = 0.0
sol_p2 = 0.2 #BYOL default
#sol_p2 = 0.0

noise_p1 = 0.0
noise_p2 = 0.0
#Noise isn't included! So we exclude for now

#Normalization?? None for now as it looks weird

resize_scale=(0.7, 1.0)
resize_ratio=(3/4, 4/3)

#Rotation isn't included in ImageNet. We just include some. 
rotate_deg = 45
rotate_p = 0.5

##Values of hps:
blur_r = (0.1,2) #like BYOL
blur_s = 13 #as ~ 128/10 like BYOL

#blur_r = (0.1,4) #like BYOL
#blur_s = 13 #as ~ 128/10 like BYOL


sol_t = 0.1 #kornia defaults. Not sure still about BYOL defaults.
sol_a = 0.1

noise_std = 0.1 

###################

###################
#Hyperparams for fine-tuning augmentations. We use: blur; jitter; grayscale (maybe) and soft augs.
blur_r_tune = blur_r
blur_s_tune = blur_s
flip_p_tune = 0.25
rotate_p_tune = 0.25
jitter_p_tune = 0.0
bw_p_tune = 0.0
blur_p_tune = 0.0

aug_pipelines_1 = get_barlow_twins_aug_pipelines(size=size,blur_r=blur_r,
                    rotate=True,jitter=True,noise=True,bw=True,blur=True,solar=True, #Whether to use aug or not
                    resize_scale=resize_scale,resize_ratio=resize_ratio,noise_std=noise_std, rotate_deg=rotate_deg,blur_s=blur_s,sol_t=sol_t,sol_a=sol_a, #hps of augs
                    flip_p=flip_p1, rotate_p=rotate_p,noise_p=noise_p1, jitter_p=jitter_p1, bw_p=bw_p1, blur_p=blur_p1,sol_p=sol_p1, #prob of performing aug
                    same_on_batch=False,stats=None, cuda=(device=='cuda'))#,P=0.0)

aug_pipelines_2 = get_barlow_twins_aug_pipelines(size=size,blur_r=blur_r,
                    rotate=True,jitter=True,noise=True,bw=True,blur=True,solar=True, #Whether to use aug or not
                    resize_scale=resize_scale,resize_ratio=resize_ratio,noise_std=noise_std, rotate_deg=rotate_deg,blur_s=blur_s,sol_t=sol_t,sol_a=sol_a, #hps of augs
                    flip_p=flip_p2, rotate_p=rotate_p,noise_p=noise_p2, jitter_p=jitter_p2, bw_p=bw_p2, blur_p=blur_p2,sol_p=sol_p2, #prob of performing aug
                    same_on_batch=False,stats=None, cuda=(device=='cuda'))#,P=0.0)

aug_pipelines = [aug_pipelines_1,aug_pipelines_2]

#show_bt_batch(dls=dls_tune,n_in=3,aug=aug_pipelines,n=10,print_augs=True)


In [ ]:
aug_pipelines_tune =  [get_barlow_twins_aug_pipelines(size=size,blur_r=blur_r_tune,
                    rotate=True,jitter=True,noise=False,bw=True,blur=True,solar=False, #Whether to use aug or not
                    resize_scale=resize_scale,resize_ratio=resize_ratio,noise_std=None, rotate_deg=rotate_deg,blur_s=blur_s_tune,sol_t=None,sol_a=None, #hps of augs
                    flip_p=flip_p_tune, rotate_p=rotate_p_tune,noise_p=0.0, jitter_p=jitter_p_tune, bw_p=bw_p_tune, blur_p=blur_p_tune,sol_p=0.0, #prob of performing aug
                    same_on_batch=False,stats=None, cuda=(device=='cuda'))]#,P=0.0)

#show_linear_batch(dls=dls_tune,n_in=3,n=10,aug=aug_pipelines_tune,print_augs=True)

In [ ]:
aug_pipelines_test = get_barlow_twins_aug_pipelines(size=size,blur_r=blur_r_tune,
                    rotate=True,jitter=True,noise=False,bw=True,blur=True,solar=False, #Whether to use aug or not
                    resize_scale=resize_scale,resize_ratio=resize_ratio,noise_std=None, rotate_deg=rotate_deg,blur_s=blur_s_tune,sol_t=None,sol_a=None, #hps of augs
                    flip_p=flip_p_tune, rotate_p=rotate_p_tune,noise_p=0.0, jitter_p=jitter_p_tune, bw_p=bw_p_tune, blur_p=blur_p_tune,sol_p=0.0, #prob of performing aug
                    same_on_batch=False,stats=None, cuda=(device=='cuda'))

In [ ]:
@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

## main training api

In [ ]:
class main_train:
    """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
    Outputs metrics on validation data"""

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 fit_policy, #policy for fitting BT
                 tune_fit_policy,
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 ):
                 store_attr()
                 self.vocab = self.dls_valid.vocab


                 #self.encoder_path = f'/content/drive/My Drive/models/baselineencoder_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'
                 #self.tuned_model_path = f'/content/drive/My Drive/models/baselinefinetuned_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'

    @staticmethod
    def fit(learn,fit_policy,epochs):

        if fit_policy == 'fit':
            learn.fit(epochs)
        
        elif fit_policy == 'fit_one_cycle':
            learn.fit_one_cycle(epochs)
       
        elif fit_policy == 'fine_tune':
            learn.fine_tune(epochs)

        else: raise Exception('Fit policy not of expected form')


    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorith, according to fit_policy"


        bt_model,encoder = create_model(which_model=self.initial_weights) #create encoder and bt model


        if self.pretrain: #train encoder according to fit policy

            learn = Learner(self.dls_train,bt_model, cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=1/self.ps,print_augs=False)])

            main_train.fit(learn,self.fit_policy,epochs=self.num_epochs)

            #torch.save(encoder.state_dict(), self.encoder_path)

        self.encoder = bt_model.encoder


    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)


        model = LinearModel(encoder=self.encoder,indim=self.indim,outdim=self.outdim) #create 'linear' model (encoder + linear head)
        
        learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        main_train.fit(learn,fit_policy=self.tune_fit_policy,epochs=self.numfit) #fine tuning (don't confuse this with fit policy!)

        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)

        predict_model(xval,yval,model,aug_pipelines_test,numavg=3)

        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        metrics['acc'] = acc
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['auc_dict'] = auc_dict
        
        #torch.save(model.state_dict(), self.tuned_model_path)
        return metrics #

    def __call__(self):

        self.train_encoder() #train (or extract) the encoder
        metrics = self.fine_tune()
        
        return metrics

if __name__ == '__main__':


    num=3
    mean_acc = 0
    for _ in range(num):
        metrics = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                    aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                    initial_weights='supervised_pretrain',pretrain=True,fit_policy='fit',tune_fit_policy='fine_tune',num_epochs=20,numfit=20, 
                    )()

        acc = metrics['acc']
        print(acc)
        mean_acc += acc


    print(mean_acc/num)




epoch,train_loss,valid_loss,time
0,5012.721191,None,00:07
1,3455.212158,None,00:07
2,2795.434326,None,00:07
3,2459.484619,None,00:07
4,2203.376221,None,00:07
5,2017.852417,None,00:07
6,1887.167358,None,00:08
7,1778.843994,None,00:07
8,1688.590942,None,00:07
9,1610.619019,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2.206946,None,00:06


epoch,train_loss,valid_loss,time
0,1.840274,None,00:06
1,1.739568,None,00:06
2,1.619048,None,00:07
3,1.498451,None,00:06
4,1.383047,None,00:06
5,1.272026,None,00:06
6,1.172073,None,00:07
7,1.083841,None,00:06
8,1.006145,None,00:06
9,0.937737,None,00:06


0.6436781883239746


...

epoch,train_loss,valid_loss,time
0,3998.411377,None,00:07
1,2783.328369,None,00:07
2,2256.380615,None,00:07
3,1956.185791,None,00:07
4,1755.180542,None,00:07
5,1625.260986,None,00:07
6,1530.836914,None,00:07
7,1447.950317,None,00:07
8,1371.629395,None,00:07
9,1305.092285,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2.155445,None,00:06


epoch,train_loss,valid_loss,time
0,1.751729,None,00:06
1,1.669657,None,00:06
2,1.579648,None,00:07
3,1.482444,None,00:07
4,1.369130,None,00:06
5,1.251022,None,00:06
6,1.146727,None,00:06
7,1.055470,None,00:06
8,0.972068,None,00:06
9,0.900794,None,00:06


0.6839080452919006
0.6570881406466166


In [ ]:
metrics['acc']

0.5747126340866089

In [ ]:
mean_acc/num

0.6570881406466166

In [ ]:
mean_acc/num

0.5747126539548238

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

InterpolationMissingOptionError: ignored